# Environment Canada weather station data

Let's dig into some weather data. After a record rainfall in B.C., it was a great opportunity to use Environment Canada's API for weather stations across Canada.

In [214]:
import pandas as pd

stations = pd.read_csv('../raw/RAW 2021 ENVIRONMENT CANADA WEATHER STATIONS.csv', encoding="latin-1", header=2)

In [4]:
active_stations = (stations[stations["Last Year"] == 2021]
                   .loc[:,"Station ID"]
                   .to_list()
                   )

This code block takes a few minutes to run. It grabs all records from 2021 for stations across Canada. If you don't want to run this over and over, you can take the results and save it into a CSV, then just read in that CSV later.

In [5]:
li = []
filepath = "../raw/RAW 2021 ENVIRONMENT CANADA ALL WEATHER STATIONS.csv"

for station_id in active_stations:
    df = pd.read_csv('https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=' + str(station_id) + '&Year=2021&timeframe=2')
    df.insert(0, "Station ID", station_id)
    li.append(df)

raw = pd.concat(li, axis=0, ignore_index=True)

### Rain at stations across Canada from Nov 11-15

Let's start by taking a look at a particularly wet stretch of days.

We'll do this by building separate dataframes for each date between Nov. 11-15. We'll use a loop to do this programmatically, so that if we ever want to do this again with more dates, we can simply add them to the list. For each iteration of the loop, we'll rename the precipation column to one with the date of the current iteration of the loop.

Finally, we put them all into a list.

In [94]:
dfs = []

for date in [11, 12, 13, 14, 15]:
    day = (raw.loc[raw["Day"] == date]
           .loc[raw["Month"] == 11]
         .rename(columns={"Total Precip (mm)": "Nov " + str(date) + " Rainfall"})
         .set_index(["Station Name"])
        )
    dfs.append(day)

Now, let's concatenate them all together and drop all the columns we don't need (anything that isn't a rainfall column we renamed in the last step).

In [227]:
rainfall = (pd.concat(dfs, axis=1)
        .filter(like="Rainfall")
        .dropna(how="all")
        .astype(float)
        )

rainfall["Total"] = rainfall.sum(axis=1)

display(rainfall.head(3))

,Nov 11 Rainfall,Nov 12 Rainfall,Nov 13 Rainfall,Nov 14 Rainfall,Nov 15 Rainfall,Total
Station Name,,,,,,
CHEMAINUS,16.4,0.0,53.4,90.0,33.8,193.6
LAKE COWICHAN,22.9,NaN,66.4,129.2,3.0,221.5
ESQUIMALT HARBOUR,6.4,2.6,12.2,29.8,60.8,111.8


### A deeper look at North Vancouver

If you sort the above table to show the stations with the most raifall, it will lead us to another interesting analysis.

In [97]:
sorted = rainfall.sort_values("Total", ascending=False)

display(sorted.head(3))

,Nov 11 Rainfall,Nov 12 Rainfall,Nov 13 Rainfall,Nov 14 Rainfall,Nov 15 Rainfall,Total
Station Name,,,,,,
N VANCOUVER WHARVES,NaN,NaN,38.8,288.2,43.0,370.0
PORT RENFREW,54.8,10.6,120.8,164.0,7.2,357.4
HOPE AIRPORT,3.1,31.1,17.1,174.0,103.5,328.8


Let's dig into the station that received the most rainfall over those days, North Vancouver Wharves.

We make another request to Environment Canada (actually, it's several requests, one for each year going back to 1980).

In [91]:
station_id = 833 ## This is the ID of the station we want to take a look at.

lis = []

for year in range(1980, 2022):
    url = 'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=' + station_id + '&Year=' + str(year) + '&Month=11&Day=14&timeframe=2'
    df = pd.read_csv(url)
    lis.append(df)

northvan_raw = pd.concat(lis, axis=0, ignore_index=True)


Then, we do a similar loop to above, where we filter by month and the two days where it seemed to receive the most rainfall: Nov. 14 and 15.

In [156]:
# Empty list for collecting dataframes for concatenation.
northvan_dfs = []

for date in [14, 15]:
    day = (northvan_raw.loc[northvan_raw["Day"] == date]
            .loc[northvan_raw["Month"] == 11]
            .set_index("Station Name")
            )
    northvan_dfs.append(day)
    
northvan_nov = pd.concat(northvan_dfs)

sums = (northvan_nov[['Year', 'Total Rain (mm)']]
           .groupby(['Year'])
           .sum()
           )

display(sums.head(3))

,Total Rain (mm)
Year,
1980,1.0
1981,22.0
1982,0.0


Now we have a very graph-able table on the rainfall in North Vancouver (and in fact, [I did graph it](https://www.datawrapper.de/_/mdLnU/)! A note, however: Environment Canada told me that there had been some issues with this station, and that a better station to use would be the Vancouver Harbour station. I repeated the above analysis using that station ID instead and charted it [here](https://www.datawrapper.de/_/pjoFW/).

### Did the rainfall on Nov. 14 and 15 in North Vancouver exceed the average rainfall for the entire month?

We can take the previous collection of records for the North Vancouver weather station and try to answer this question. This time, we group by month only, rather than month and day.

In [224]:
northvan_november = (northvan_raw
                   .loc[northvan_raw["Month"] == 11]
                   .loc[:,["Year", "Total Rain (mm)"]]
                   .groupby(["Year"])
                   .sum()
                   )

display(northvan_november.head(5))

,Total Rain (mm)
Year,
1980,372.2
1981,246.0
1982,0.0
1983,412.2
1984,412.9


Now, we calculate the average.

In [155]:
avg = northvan_november["Total Rain (mm)"].mean()
two_days = sums.loc[2021,"Total Rain (mm)"]

display("Average November rainfall: {0:.1f} mm".format(avg))
display("Rainfall on Nov. 14 and 15: {0:.1f} mm".format(two_days))

'Average November rainfall: 252.3 mm'

'Rainfall on Nov. 14 and 15: 331.2 mm'

The answer to our question is yes: the rainfall in those two days exceeded the average November rainfall for that station.

### Mapping rainfall in Nova Scotia on Nov. 23/24.

Two provinces were hammered with heavy storms on Nov. 23 and 24: Nova Scotia and Newfoundland and Labrador. Let's take a closer look at those two provinces.

We'll use a block of code similar to the one above that makes calls to Environment Canada's API. However, we'll vary station ID and iterate through a list of station IDs in each province.

In [253]:
provinces = ["NOVA SCOTIA", "NEWFOUNDLAND"]

province_data = []

# Loop that runs for each province in the list above.
for province in provinces:
    
    # Gets a list of stations in that province.
    station_list = (stations
                    .loc[stations["Province"] == province]
                    .loc[:,"Station ID"]
                    .dropna()
                    .to_list()
                    )
    
    # Empty list to put station data into.
    li = []

    # Loop that runs for each station in the province we're looking at.
    for station_id in station_list:
        url = 'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=' + str(station_id) + '&Year=2021&timeframe=2'
        df = (
            pd.read_csv(url)
            )
        li.append(df)
    
    # Bring together all station data to a dataframe for the entire province.
    province_raw = pd.concat(li, axis=0, ignore_index=True)
    
    # Add all province data to a list keeping track of dataframes for all provinces.  
    province_data.append(province_raw)

Now that we have all our station data from both provinces, we can play with it a bit to show us what we're really interested: rainfall on Nov. 23 and 24.

In [254]:
for df in province_data:
        data = (df
                .loc[df.loc[:,"Day"].isin([23, 24])]
                .loc[df["Month"] == 11]
                .loc[df["Year"] == 2021]
                .pivot(columns="Day", values="Total Precip (mm)", index=["Climate ID", "Station Name", "Longitude (x)", "Latitude (y)"])
                .dropna(how="all")
                .reset_index()
                .set_index("Station Name")
                .drop(columns=["Climate ID"])
                .rename(columns={23: "Nov. 23 Rainfall", 24: "Nov. 24 Rainfall"})
                )
        
        # Create column to sum both days and sort on it.
        data["Total"] = data["Nov. 23 Rainfall"] + data["Nov. 24 Rainfall"]
        data = data.sort_values("Total", ascending=False)
        
        display(data.head(10))

Day,Longitude (x),Latitude (y),Nov. 23 Rainfall,Nov. 24 Rainfall,Total
Station Name,,,,,
INGONISH BEACH RCS,-60.41,46.66,232.8,33.2,266.0
SYDNEY A,-60.05,46.16,117.8,45.4,163.2
NORTHEAST MARGAREE (AUT),-60.98,46.37,103.1,13.5,116.6
PORT HAWKESBURY,-61.37,45.66,102.1,5.0,107.1
LOUISBOURG,-59.97,45.92,72.8,27.8,100.6
COLLEGEVILLE AUTO,-62.01,45.49,97.0,0.7,97.7
ESKASONI FIRST NATION AUTOMATIC WEATHER STATION,-60.65,45.92,58.1,17.9,76.0
CHETICAMP HIGHLANDS NATIONAL PARK,-60.95,46.65,53.5,18.0,71.5
SABLE ISLAND A,-59.96,43.93,43.8,20.2,64.0


Day,Longitude (x),Latitude (y),Nov. 23 Rainfall,Nov. 24 Rainfall,Total
Station Name,,,,,
PORT AUX BASQUES,-59.15,47.57,68.1,97.0,165.1
BURNT POND,-57.33,48.17,54.0,0.0,54.0
CARTWRIGHT A,-57.04,53.68,14.8,31.0,45.8
HAPPY VALLEY GOOSE BAY,-60.56,53.41,16.9,24.8,41.7
GOOSE A,-60.42,53.32,19.4,22.2,41.6
MARY'S HARBOUR A,-55.85,52.30,17.3,22.5,39.8
FEROLLE POINT (AUT),-57.10,51.02,11.2,11.3,22.5
STEPHENVILLE RCS,-58.57,48.56,5.7,16.7,22.4
ST. ANTHONY A,-56.07,51.39,12.5,9.0,21.5


This code outputs tables that can then be mapped (I did maps for both [Nova Scotia](https://www.datawrapper.de/_/uu0Ld/) and [Newfoundland and Labrador](https://www.datawrapper.de/_/Sd2Ez/)).

\-30\-